## Criando uma rede neural convolucional
***

Neste tutorial iremos criar uma rede neural convolucional para detecção de gatos e cachorros

***

In [1]:
from keras.models import Sequential, model_from_json
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

Using TensorFlow backend.


In [2]:
classificador = Sequential()

In [3]:
qtd_filtros = 32 # Recomendado é 64
dimensoes_do_kernel = (3, 3) # matriz 3x3
dimensoes_da_imagem = (64, 64, 3) # (largura, altura, canais rgb)
classificador.add(Conv2D(
    qtd_filtros,
    dimensoes_do_kernel,
    input_shape = dimensoes_da_imagem,
    activation = 'relu'
))

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
# Vai acelerar o processamento, ou seja, vai pegar o mapa de caracteristicas
# e vai normaliza os valores em uma escala entre 0 e 1
classificador.add(BatchNormalization())

In [5]:
# Usado para pegar o maior valor do mapa de caracteristicas, realçando-as.
matriz_de_pooling = (2, 2) # matriz 2x2
classificador.add(MaxPooling2D(pool_size = matriz_de_pooling))

In [6]:
# Adicionando outra camada de convolução
classificador.add(Conv2D(
    qtd_filtros,
    dimensoes_do_kernel,
    input_shape = dimensoes_da_imagem,
    activation = 'relu'
))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = matriz_de_pooling))

In [7]:
# Adicionar o flattening para vetorizar a matriz para entrar na rede neural densa
classificador.add(Flatten())

In [8]:
# Vamos criar a primeira camada da rede neural densa
classificador.add(Dense(
    units = 128,
    activation = 'relu'
))

In [9]:
# Vamos zerar 20% dos valores de entrada para evitar o overfitting
classificador.add(Dropout(0.2))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
# Vamos criar a primeira camada oculta da rede neural densa
classificador.add(Dense(
    units = 128,
    activation = 'relu'
))

In [11]:
# Vamos zerar 20% dos valores de entrada para evitar o overfitting
classificador.add(Dropout(0.2))

In [12]:
# Vamos criar a camada de saída da rede neural densa
classificador.add(Dense(
    units = 1,
    activation = 'sigmoid'
))

In [13]:
# Vamos compilar a rede neural
classificador.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

***

In [14]:
# Cria novas imagens a partir das imagens existentes
normalizacao = 1./255
# ROTATION_RANGE = Grau de rotação da imagem
# HORIZONTAL_FLIP = Vai fazer giros horizontais nas imagens
# SHEAR_RANGE = Faz a mudança dos pixels para outra direção
# HEIGHT_SHIFT_RANGE = Faixa de mudança da altura da imagem
# ZOOM_RANGE = Faixa de mudança do zoom da imagem
gerador_de_imagens = ImageDataGenerator(
    rescale = normalizacao,
    rotation_range = 7,
    horizontal_flip = True,
    shear_range = 0.2,
    height_shift_range = 0.07,
    zoom_range = 0.2
)

In [15]:
# Cria novas imagens de teste a partir das imagens existentes
gerador_de_imagens_teste = ImageDataGenerator(rescale=normalizacao)

In [16]:
# Criar a base de dados de treinamento
# TARGET_SIZE = Tamanho das imagens
# BATCH_SIZE = De quantas em quantas imagens será feito o treinamento
# CLASS_MODE = Tipo de saída, no caso é binario
base_treinamento = gerador_de_imagens.flow_from_directory(
    'dataset/training_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 4000 images belonging to 2 classes.


In [17]:
# Criar a base de dados de teste
base_teste = gerador_de_imagens_teste.flow_from_directory(
    'dataset/test_set',
    target_size = (64, 64),
    batch_size = 32,
    class_mode = 'binary'
)

Found 1000 images belonging to 2 classes.


In [18]:
# Fazer o treinamento
# STEPS_PER_EPOCH = Quantidade de imagens que será treinada por ciclo, ideal o total de imagens
# EPOCHS = Quantidade de ciclos que será treinado, quanto maior melhor
# VALIDATION_DATA = Imagens de teste
# VALIDATION_STEPS = Quantidade de imagens de teste por ciclo
classificador.fit_generator(
    base_treinamento,
    steps_per_epoch = 4000/32,
    epochs = 5,
    validation_data = base_teste,
    validation_steps = 1000/32
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
125/125 [==============================] - 36s 285ms/step - loss: 0.8190 - acc: 0.5870 - val_loss: 0.8012 - val_acc: 0.5880
Epoch 2/5
125/125 [==============================] - 36s 290ms/step - loss: 0.6702 - acc: 0.6250 - val_loss: 0.7039 - val_acc: 0.5880
Epoch 3/5
125/125 [==============================] - 39s 311ms/step - loss: 0.6154 - acc: 0.6598 - val_loss: 0.6440 - val_acc: 0.6060
Epoch 4/5
125/125 [==============================] - 35s 282ms/step - loss: 0.6005 - acc: 0.6733 - val_loss: 0.5899 - val_acc: 0.6800
Epoch 5/5
125/125 [==============================] - 33s 266ms/step - loss: 0.5773 - acc: 0.6950 - val_loss: 0.8552 - val_acc: 0.5280


In [19]:
# Salvando a estrutura da rede neural
classificador_json = classificador.to_json()
with open('classificador.json', 'w') as json_file:
    json_file.write(classificador_json)

In [20]:
# Salvando os pesos da rede neural (pip install h5py)
classificador.save_weights('classificador.h5')

***

In [21]:
# Pegando a estrutura da rede neural
with open('classificador.json', 'r') as json_file:
    estrutura = json_file.read()

In [22]:
# Criando o classificador
classificador_externo = model_from_json(estrutura)

In [23]:
# Pegando os pesos da rede neural
classificador_externo.load_weights('classificador.h5')

In [24]:
imagem_teste = image.load_img('dataset/test_set/gato/cat.3500.jpg', target_size = (64, 64))

In [25]:
imagem_teste_array = image.img_to_array(imagem_teste)

In [26]:
# Normalização
imagem_teste_array /= 255

In [27]:
# Formatar para o formato do tensorFlow (qtd, largura, altura, canais)
imagem_teste_array = np.expand_dims(imagem_teste_array, axis = 0)

In [28]:
previsao = classificador_externo.predict(imagem_teste_array)

In [29]:
print(base_treinamento.class_indices)

{'gato': 1, 'cachorro': 0}


In [30]:
# 0 < previsao < 0.5 = Cachorro
# 0.5 <= previsao < 1 = Gato
print(previsao)
if previsao >= 0.5:
    print("É um gato")
else:
    print("É um cachorro")

[[0.54427975]]
É um gato
